# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import time
import gc
import os

# Load datasets
Load both the 30s and 3s features.

In [ ]:
# Load datasets
print("Loading datasets...")
file_path_30sec = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv"
file_path_3sec = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv"

df_30sec = pd.read_csv(file_path_30sec)
print(f"30-sec dataset loaded with shape: {df_30sec.shape}")

# Try loading the 3-sec dataset if it exists
try:
    df_3sec = pd.read_csv(file_path_3sec)
    print(f"3-sec dataset loaded with shape: {df_3sec.shape}")
    use_combined = True
except:
    print("3-sec dataset not found or couldn't be loaded. Using only 30-sec dataset.")
    use_combined = False


# Preprocessing
Includes merging the 30s and 3s features into a single dataset. To implement this, we used a **label encoder** and scaled the data appropiately to avoid large differences between the 30s features and the 3s features.

In [ ]:
# Data preparation function
def prepare_data(df):
    # Drop filename column if it exists
    if 'filename' in df.columns:
        df = df.drop(columns=['filename'])
    
    # Separate features and labels
    X = df.drop(columns=['label']).values
    y = df['label'].values
    
    return X, y

In [ ]:
# Prepare datasets
print("Preparing 30-sec dataset...")
X_30sec, y_30sec = prepare_data(df_30sec)
print(f"30-sec data: X shape: {X_30sec.shape}, y shape: {y_30sec.shape}")

# Encode labels consistently (fit on 30sec labels first to ensure consistency)
print("Encoding labels...")
label_encoder = LabelEncoder()
y_30sec_encoded = label_encoder.fit_transform(y_30sec)
num_classes = len(label_encoder.classes_)
print(f"Number of classes: {num_classes}")
print(f"Classes: {label_encoder.classes_}")

# Initialize scalers
scaler_30sec = StandardScaler()

# Scale features
print("Standardizing 30-sec features...")
X_30sec_scaled = scaler_30sec.fit_transform(X_30sec)

if use_combined:
    print("Preparing 3-sec dataset...")
    X_3sec, y_3sec = prepare_data(df_3sec)
    print(f"3-sec data: X shape: {X_3sec.shape}, y shape: {y_3sec.shape}")
    
    # Use the same encoder for 3-sec labels
    y_3sec_encoded = label_encoder.transform(y_3sec)
    
    # Check if feature dimensions match
    if X_3sec.shape[1] != X_30sec.shape[1]:
        print(f"Warning: Feature dimensions don't match! 30-sec: {X_30sec.shape[1]}, 3-sec: {X_3sec.shape[1]}")
        print("Using only 30-sec dataset")
        use_combined = False
    else:
        # Scale 3-sec features using 30-sec scaler
        print("Standardizing 3-sec features...")
        X_3sec_scaled = scaler_30sec.transform(X_3sec)
        
        # Combine datasets
        print("Combining datasets...")
        X_combined = np.vstack([X_30sec_scaled, X_3sec_scaled])
        y_combined_encoded = np.concatenate([y_30sec_encoded, y_3sec_encoded])
        print(f"Combined dataset: {X_combined.shape} features, {len(y_combined_encoded)} labels")

# Set input dimension and data to use
input_dim = X_30sec_scaled.shape[1]
if use_combined:
    X_data = X_combined
    y_data = y_combined_encoded
    print("Using combined dataset for training")
else:
    X_data = X_30sec_scaled
    y_data = y_30sec_encoded
    print("Using only 30-sec dataset for training")

# Split data
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42, stratify=y_data
)
print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")

# Convert to categorical for loss function compatibility
print("Converting labels to categorical...")
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes)
print("Conversion complete")


# Model Architecture
Includes a function to define model with self-attention and residual connections

In [ ]:
optimizers = {
    "Adam": keras.optimizers.Adam(learning_rate=0.001),
    "SGD": keras.optimizers.SGD(learning_rate=0.004, momentum=0.9),
    'RMSprop': keras.optimizers.RMSprop(learning_rate=0.001)
}

models = {}

def build_advanced_model(input_dim, num_classes, optimizer):
    # Input layer
    inputs = keras.Input(shape=(input_dim,))
    
    # Self-attention mechanism
    # Reshape for attention
    attention_input = keras.layers.Reshape((input_dim, 1))(inputs)
    
    # Multi-head attention layer
    attention = keras.layers.MultiHeadAttention(
        num_heads=4, key_dim=4
    )(attention_input, attention_input)
    
    # Reshape back to original dimensions
    attention = keras.layers.Reshape((input_dim,))(attention[:, :, 0])
    
    # Combine original input with attention features
    x = keras.layers.Concatenate()([inputs, attention])
    
    # First dense block
    x = keras.layers.Dense(512, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.4)(x)
    
    # First residual block
    residual = x
    x = keras.layers.Dense(512, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(512, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.add([x, residual])  # Skip connection
    
    # Second dense block
    x = keras.layers.Dense(256, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    
    # Second residual block
    residual = x
    x = keras.layers.Dense(256, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(256, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.add([x, residual])  # Skip connection
    
    # Third dense block
    x = keras.layers.Dense(128, activation='selu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.2)(x)
    
    # Output layer
    outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

for name, optimizer in optimizers.items():
    models[name] = build_advanced_model(input_dim, num_classes, optimizer)
    models[name].summary()
    



# Mixing and splitting data into training and test set


In [ ]:
# Define mixup data augmentation
def mixup_data(x, y, alpha=0.2):
    """Performs mixup augmentation on the batch."""
    batch_size = len(x)
    weights = np.random.beta(alpha, alpha, batch_size)
    
    # Reshape weights to allow broadcasting
    weights = weights.reshape(batch_size, 1)
    
    # Create pairs of samples
    index = np.random.permutation(batch_size)
    x1, x2 = x, x[index]
    y1, y2 = y, y[index]
    
    # Generate mixed samples
    x_mixed = x1 * weights + x2 * (1 - weights)
    y_mixed = y1 * weights + y2 * (1 - weights)
    
    return x_mixed, y_mixed

# Custom training generator with mixup
class MixupGenerator(keras.utils.Sequence):
    def __init__(self, x, y, batch_size=32, alpha=0.2, shuffle=True):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.indices = np.arange(len(x))
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))
    
    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[batch_indices]
        batch_y = self.y[batch_indices]
        
        # Apply mixup
        batch_x, batch_y = mixup_data(batch_x, batch_y, self.alpha)
        
        return batch_x, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Use mixup generator for training
print("Setting up mixup generator...")
train_generator = MixupGenerator(
    X_train, y_train_cat, batch_size=32, alpha=0.2
)


# Training model
This model will be using callback functions: EarlyStopping and ReduceLROnPlateau

In [ ]:
import pickle
results = {}
for name, model in models.items():
    early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True,
    verbose=1
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=0.00001,
        verbose=1
    )
    history = model.fit(
        train_generator,
        epochs=100,     # More epochs for better convergence
        validation_data=(X_test, y_test_cat),
        callbacks=[early_stopping, reduce_lr],
        verbose=1       # Progress bar
    )

    results[name] = history
    # Save history
    with open(f'/kaggle/working/{name}_history.pkl', 'wb') as file:
        pickle.dump(history.history, file)
    
    # Save model
    model.save(f'/kaggle/working/{name}_model.h5')
    print(f"Model and history for {name} saved successfully")

# Model Evaluation

In [ ]:
def plot_optimizer_comparison(histories_dict):
    # Dictionary to store reformatted histories
    reformatted_histories = {}
    
    # Convert the history objects to the format expected by the original function
    for name, history_obj in histories_dict.items():
        reformatted_histories[name] = {'history': history_obj.history}
    
    # Continue with your original function logic using the reformatted data
    # Step 2: Plot training and validation metrics
    plt.figure(figsize=(14, 5))
    
    # Plot 1: Validation Accuracy
    plt.subplot(1, 2, 1)
    for name, metrics in reformatted_histories.items():
        plt.plot(metrics['history']['val_accuracy'], label=f"{name} Val Acc")
    plt.title('Validation Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot 2: Validation Loss
    plt.subplot(1, 2, 2)
    for name, metrics in reformatted_histories.items():
        plt.plot(metrics['history']['val_loss'], label=f"{name} Val Loss")
    plt.title('Validation Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_optimizer_comparison(results)

In [ ]:
# Evaluate models

def evaluate_model(model):
    try:
        print(f"Evaluating FCNN model with optimizer: {name}")
        test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=1)
        print(f"Test accuracy: {test_acc:.4f}")
    except Exception as e:
        print(f"Error during evaluation: {e}")
    
    # Generate predictions, compute evaluation report and confusion matrix
    try:
        print("Generating predictions...")
        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test_cat, axis=1)

        print("\nClassification Report:")
        report = classification_report(y_test_classes, y_pred_classes, 
                                      target_names=label_encoder.classes_)
        print(report)
        
        cm = tf.math.confusion_matrix(y_test_classes, y_pred_classes).numpy()
        
        # Plot confusion matrix
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=label_encoder.classes_,
                   yticklabels=label_encoder.classes_)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix for FCNN with {name}')
        plt.tight_layout()
        plt.savefig(f'fcnn_{name}_confusion_matrix.png')
        
        # Calculate per-genre accuracy
        genre_acc = {}
        for i, genre in enumerate(label_encoder.classes_):
            genre_indices = y_test_classes == i
            if np.sum(genre_indices) > 0:  # Avoid division by zero
                genre_acc[genre] = np.mean(y_pred_classes[genre_indices] == i)
        
        # Print per-genre accuracy
        print("\nPer-genre accuracy:")
        for genre, acc in genre_acc.items():
            print(f"{genre}: {acc:.3f}")

    except Exception as e:
        print(f"Error during prediction/visualization: {e}")
        
        
        
# Save the scaler for later use
import joblib
joblib.dump(scaler_30sec, 'feature_scaler.joblib')
joblib.dump(label_encoder, 'label_encoder.joblib')
print("Scaler and encoder saved")
        

    
    

In [ ]:
for name, model in models.items():
    evaluate_model(model)